## Data and Package Import

In [8]:
import pyodbc
import pandas as pd


def get_connQuery():
    return pyodbc.connect("DRIVER={ODBC Driver 17 for SQL Server};"
                          "SERVER=DESKTOP-FK7HVAL\PRACTICEDB;"
                          "DATABASE=Messy;"
                          "Trusted_Connection=yes;")

def get_SqlQuery():
    connQuery = get_connQuery()
    return pd.read_sql_query(
        '''
    SELECT A.* FROM Messy.dbo.AllCurrentData A with (NOLOCK)   
    ''', connQuery)

SQLQuery = get_SqlQuery()
AllCars = pd.DataFrame(SQLQuery, columns=['JasonID', 'VIN', 'Price','odometer','Year','Make','Model','VehicleConfig'
                                          ,'color','Damage','Usage','PrevOwners','Transmission','EngName'])


In [12]:
# AllCars.describe()
# AllCars.info()
AllCars.head()

,JasonID,VIN,Price,odometer,Year,Make,Model,VehicleConfig,color,Damage,Usage,PrevOwners,Transmission,EngName
0,1,2G1WH55K6Y9264263,1200.0,206223.0,2000,chevrolet,impala,LS 4dr Sedan,Black,0,Personal Use,4,AUTOMATIC,6 Cyl
1,2,2G1WF55E9Y9184102,595.0,218640.0,2000,chevrolet,impala,4dr Sedan,Bright White,0,Rental Use,5,AUTOMATIC,6 Cyl
2,3,2G1WH55K2Y9364912,2400.0,197374.0,2000,chevrolet,impala,LS 4dr Sedan,Light Driftwood Metallic,0,Personal Use,4,AUTOMATIC,6 Cyl
3,5,2G1WF52E419238190,1990.0,99651.0,2001,chevrolet,impala,4dr Sedan (3.4L 6cyl 4A),Sandrift Metallic,1,Personal Use,4,AUTOMATIC,6 Cyl
4,6,2G1WH55K819164882,1995.0,160000.0,2001,chevrolet,impala,LS 4dr Sedan (3.8L 6cyl 4A),Galaxy Silver Metallic,0,Corporate Fleet,4,AUTOMATIC,6 Cyl


## Modeling

In [41]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [60]:
X = AllCars.iloc[:,3:5] # only miles year
y = AllCars.iloc[:,2]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 5)

In [62]:
regressor = xgb.XGBRegressor(
    n_estimators=100,
    reg_lambda=1,
    gamma=0,
    max_depth=3
)
regressor.fit(X_train, y_train)

C:\Users\Quinn\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[22:53:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [63]:
print(pd.DataFrame(regressor.feature_importances_))

          0
0  0.056851
1  0.943149


In [39]:
y_pred = regressor.predict(X_test)
print(y_pred)

[18349.904 23908.904 11444.933 ... 10028.363 17544.941 22499.064]


In [64]:
mean_squared_error(y_test, y_pred)

49930984.70728027

In [59]:
print(X_test.shape)
print(y_test.shape)
X.head()

(35770, 1)
(35770,)


,odometer
0,206223.0
1,218640.0
2,197374.0
3,99651.0
4,160000.0


In [51]:
Scored_test = X_test.append(y_test)

MemoryError: 

In [ ]:
Scored_test.head)